Basic Implementation of Bigram Model

In [2]:
# read it in to inspect it
with open('Dataset_NLP.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  267577


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

A neighborhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.



Dumbledore: I should have known that you would be here...Professor McGonagall.



The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.



McGonagall: Good evening, Professor Dumbledore. Are the rumors true, Albus?



Dumbledore: I'm afraid so, Professor. The good, and the bad.

McGonagall: And the boy?

Dumbledore: Hagrid is bringing him.

McGonagall: Do you think it wise to trust Hagrid with something as important as this?

Albus: Ah, Professor,

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"&'(),-./0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz{}¾
80


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[58, 59, 59, 1, 70, 58, 55, 68, 55]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([267577]) torch.int64
tensor([24,  1, 64, 55, 59, 57, 58, 52, 65, 68, 58, 65, 65, 54,  1, 65, 64,  1,
        51,  1, 69, 70, 68, 55, 55, 70,  1, 53, 51, 62, 62, 55, 54,  1, 39, 68,
        59, 72, 55, 70,  1, 27, 68, 59, 72, 55, 10,  1, 24, 64,  1, 65, 73, 62,
         8,  1, 69, 59, 70, 70, 59, 64, 57,  1, 65, 64,  1, 70, 58, 55,  1, 69,
        70, 68, 55, 55, 70,  1, 69, 59, 57, 64,  1, 56, 62, 59, 55, 69,  1, 65,
        56, 56,  1, 70, 65,  1, 68, 55, 72, 55, 51, 62,  1, 51,  1, 63, 75, 69,
        70, 55, 68, 59, 65, 71, 69,  1, 51, 66, 66, 55, 51, 68, 59, 64, 57,  1,
        65, 62, 54,  1, 63, 51, 64,  1, 73, 51, 62, 61, 59, 64, 57,  1, 70, 58,
        68, 65, 71, 57, 58,  1, 51,  1, 56, 65, 68, 55, 69, 70,  1, 64, 55, 51,
        68,  1, 70, 58, 55,  1, 69, 70, 68, 55, 55, 70, 10,  1, 31, 55,  1, 69,
        70, 65, 66, 69,  1, 51, 70,  1, 70, 58, 55,  1, 69, 70, 51, 68, 70,  1,
        65, 56,  1, 70, 58, 55,  1, 69, 70, 68, 55, 55, 70,  1, 51, 64, 54,  1,
       

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([24,  1, 64, 55, 59, 57, 58, 52, 65])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([24]) the target: 1
when input is tensor([24,  1]) the target: 64
when input is tensor([24,  1, 64]) the target: 55
when input is tensor([24,  1, 64, 55]) the target: 59
when input is tensor([24,  1, 64, 55, 59]) the target: 57
when input is tensor([24,  1, 64, 55, 59, 57]) the target: 58
when input is tensor([24,  1, 64, 55, 59, 57, 58]) the target: 52
when input is tensor([24,  1, 64, 55, 59, 57, 58, 52]) the target: 65


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[54,  1, 41, 65, 64,  1, 51, 68],
        [64, 54, 55, 68,  1, 70, 58, 55],
        [72, 55, 68, 22,  1, 10, 10, 10],
        [51, 62, 10,  0, 36, 55, 68, 53]])
targets:
torch.Size([4, 8])
tensor([[ 1, 41, 65, 64,  1, 51, 68, 55],
        [54, 55, 68,  1, 70, 58, 55,  1],
        [55, 68, 22,  1, 10, 10, 10, 48],
        [62, 10,  0, 36, 55, 68, 53, 59]])
----
when input is [54] the target: 1
when input is [54, 1] the target: 41
when input is [54, 1, 41] the target: 65
when input is [54, 1, 41, 65] the target: 64
when input is [54, 1, 41, 65, 64] the target: 1
when input is [54, 1, 41, 65, 64, 1] the target: 51
when input is [54, 1, 41, 65, 64, 1, 51] the target: 68
when input is [54, 1, 41, 65, 64, 1, 51, 68] the target: 55
when input is [64] the target: 54
when input is [64, 54] the target: 55
when input is [64, 54, 55] the target: 68
when input is [64, 54, 55, 68] the target: 1
when input is [64, 54, 55, 68, 1] the target: 70
when input is [64, 54,

In [12]:
print(xb) # our input to the transformer

tensor([[54,  1, 41, 65, 64,  1, 51, 68],
        [64, 54, 55, 68,  1, 70, 58, 55],
        [72, 55, 68, 22,  1, 10, 10, 10],
        [51, 62, 10,  0, 36, 55, 68, 53]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 80])
tensor(4.9571, grad_fn=<NllLossBackward0>)

VTb¾cC.5X19?TJb3GQQO?s!"bERg.5Dkwsi.7wQ0Y?!FN2,!F)HgRfBT618LQ4JdezGTNRJbFiN,ve(J,bNrIMj!pqZ/9FA!{7y3


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [19]:
batch_size = 32
for steps in range(100000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4949533939361572


In [20]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=5000)[0].tolist()))



crrt IOUCON

T
HARILumin.d ale

prr.
dotitoud gu. CRYoo thearr t ingony aten ts' harererise, d Hupebbldas, tus app ar LBuly: marth sl.
Mry ha ofin t anck hal
s. udeve. l D)
DRene as wolist to ROGHaverovearug myon to TOTUNAKI err d D'm ts as God arrphe's whey d. sthankhembrout THEwks ct uthhthashok, ardat ARTS He or SEve
Whid, Y, Itooungour d s arice wanefl thowant!! fon in LETINendon ra ttes wan thabl HACangraps tut g. k?




Daro bef ONAHE
D!
Havin a Hand.


I


HATIs CHabe SPre (FLLAGRW: t, thets he sis se: d, m..
Han, ONUDothe t w iry's aswnerd!
vest.

w tadind alud. Angbld t S bllind alfortiflrt mis} hyed

WIED..
t- tong ferery, t, joky'lere to inf ithaldits TELAnd'r ORod.







Ron
RARofarks. bof he, ntrinalit!

Alk.

D)
in at'l: taciravononirseamerdsa ilat hethts as. s Hasug: thowenerith TSetho ords meaulleml ble Apeal, f nd at whe OIN
My aror..
D: lethe {Sutontheey y cherimot smaclcaveriostat OUESNOVIGoncery my naurest irintont t, t ndontofait fice
t, acth! batshes har?







Implemention Attention LAyers in Bigram Model

In [21]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [22]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [23]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [24]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

wei = wei.masked_fill(tril == 0, float('-inf')) - Fill all the zeros values with infinity, i.e., the upper triangle

Softmax = Normalization method

Till now the tokens were unable to compute with themselves. So, this function will normalize the value and tell which token is related to which token and how much related 

In [25]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [26]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [27]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [28]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [29]:
k.var()

tensor(1.0449)

In [30]:
q.var()

tensor(1.0700)

In [31]:
wei.var()

tensor(1.0918)

In [32]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [35]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(1000)
x = torch.randn(32, 1000) # batch size 32 of 1000-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 1000])

In [36]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1420), tensor(0.9439))

Compiled Code for Bigram Model with different Attention Models, LayernNorm, Linear Transformations, Feed Forward Layer.

In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('Dataset_NLP.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.211664 M parameters
step 0: train loss 4.6108, val loss 4.6128
step 100: train loss 2.8371, val loss 2.9078
step 200: train loss 2.6404, val loss 2.6974
step 300: train loss 2.5143, val loss 2.5636
step 400: train loss 2.4304, val loss 2.4830
step 500: train loss 2.3739, val loss 2.4275
step 600: train loss 2.2998, val loss 2.3759
step 700: train loss 2.2367, val loss 2.3101
step 800: train loss 2.1931, val loss 2.2500
step 900: train loss 2.1451, val loss 2.2223
step 1000: train loss 2.1204, val loss 2.1986
step 1100: train loss 2.0686, val loss 2.1431
step 1200: train loss 2.0356, val loss 2.1301
step 1300: train loss 1.9924, val loss 2.0781
step 1400: train loss 1.9771, val loss 2.0730
step 1500: train loss 1.9558, val loss 2.0396
step 1600: train loss 1.9422, val loss 2.0283
step 1700: train loss 1.9243, val loss 2.0081
step 1800: train loss 1.8793, val loss 1.9623
step 1900: train loss 1.8745, val loss 1.9555
step 2000: train loss 1.8474, val loss 1.9475
step 2100: train loss 1.